## Libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Dependencies and Setup
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
# Go to existing database with automap_base
from sqlalchemy.ext.automap import automap_base
# Work through mapper to use python code
from sqlalchemy.orm import Session, relationship
# Inspect with python
from sqlalchemy import create_engine, inspect
from config import APIKEY

In [ ]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [ ]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

-------------

## Open Saved JSON

In [ ]:
with open('../../datasets/top_2020_yil_5.json') as json_file:
    data = json.load(json_file)

In [ ]:
all_logs = pd.read_json(data).copy()
all_logs

In [ ]:
# Create Weather and Earthquake Classes
# ----------------------------------
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

In [ ]:
# Use this to clear out the db
# ----------------------------------
# # Session.rollback(self)
# Base.metadata.drop_all(engine)
# session.commit()

In [ ]:
## Class base template to upload to sqlite
class top2020Yil5(Base):
    __tablename__ = 'top_2020_yil_5'
 
    id = Column(Integer, primary_key=True)
    POSITION = Column(String(3))
    PLAYER_NAME = Column(String(70))
    YEAR = Column(Float)
    YIL = Column(Float)
    ROUND_NUMBER = Column(Float)
    OVERALL_PICK = Column(Float)
    MIN_PRED = Column(Float)
    PER = Column(Float)
    INJURY_PY = Column(Float)
    INFLATION = Column(Float)
    SAL_PRED = Column(Float)
    SAL_RISIDUAL = Column(Float)


In [ ]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
# Engine opens the door. Conn is the walk through sign
engine = create_engine("sqlite:///db.sqlite")
conn = engine.connect()
# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# Create a Session Object to Connect to DB
# ----------------------------------
session = Session(bind=engine)

In [ ]:
session.add(top_2020_yil_5)

In [ ]:
session.commit()

In [ ]:
# # Create Database Connection
# # ----------------------------------
# # Creates a connection to our DB
# # Engine opens the door. Conn is the walk through sign
# engine = create_engine("sqlite:///db.sqlite")
# conn = engine.connect()
# # Create a "Metadata" Layer That Abstracts our SQL Database
# # ----------------------------------
# # Create (if not already in existence) the tables associated with our classes.
# Base.metadata.create_all(engine)
# # Create a Session Object to Connect to DB
# # ----------------------------------
# session = Session(bind=engine)

In [ ]:
## Function: Add Weather Series to sql
# def addToSQL(all_logs):
#     x=0
#     while x <= (len(all_logs) - 1):
#         try: 
#             weather_data = WeatherSeries(
#                 city = all_logs["city"][x],
#                 country = all_logs["country"][x],
#                 region = all_logs["region"][x],
#                 avgtemp = all_logs["avgtemp"][x],
#                 date = all_logs["date"][x],
#                 date_epoch = all_logs["date_epoch"][x],
#                 maxtemp = all_logs["maxtemp"][x],
#                 mintemp = all_logs["mintemp"][x],
#                 sunhour = all_logs["sunhour"][x],
#                 totalsnow = all_logs["totalsnow"][x],
#                 uv_index = all_logs["uv_index"][x],
#                 magnitude = all_logs["magnitude"][x],
#                 place = all_logs["place"][x],
#                 lat = all_logs["lat"][x],
#                 long = all_logs["long"][x],
#                 )
#             x+=1
#         except OperationalError:
#             pass
            
#         # Add Records to the Appropriate DB
#         # ----------------------------------
#         # Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
#         session.add(weather_data)
#         # session.add(earthquake_data)
#         session.commit()
#     return 'Complete: Uploaded to SQLite DB'

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
# Check db table names
Base.classes.keys()
# weather_table = Base.classes.top2020Yil5
# weather_container = session.query(top_2020_yil_5).limit(10)
# weather_data = []

------------